In [1]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score
from sklearn.utils import shuffle
from sklearn import model_selection

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('term_dataset/1_n.txt', 'r') as f:
    content_n = f.read()

with open('term_dataset/1_y.txt', 'r') as f:
    content_y = f.read()

content_y

'Массив представляет собой структуру данных, которая предназначена для хранения элементов, как правило, одного типа. Массив может состоять из целых чисел, вещественных, может состоять из символов, а также, например, из строк. Второй важный термин это индекс элемента массива. Итак, первое это смещение элемента относительно начала массива, то есть насколько позиции элемент смещен относительно его левой границы. Ну и вторая формулировка наиболее простая это просто позиция элемента в массиве. Первая характеристика это то, что массив должен иметь имя. Еще его называют идентификатор. С помощью имени мы осуществляем обращение к элементам массива. Если нам нужно изменить элемент или же просто его прочитать, то мы используем операцию обращения к элементам по индексу. Также, друзья, эту операцию называют индексация. Друзья, элементы в массиве располагаются последовательно, то есть в памяти они занимают ячейку за ячейкой. Ну и в заключение добавим, что массивы имеют конечный размер, поскольку объ

In [3]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str):
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead 
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences

### dataset

In [4]:
negative = split_into_sentences(content_n)
positive = split_into_sentences(content_y)

In [21]:
df = pd.DataFrame(columns=['target','text'])
df = pd.concat([pd.DataFrame({'text': negative, 'target': np.zeros(len(negative), dtype=int)}),
               pd.DataFrame({'text': positive, 'target': np.ones(len(positive), dtype=int)})], ignore_index=True)
df

,text,target
0,"Друзья, привет!",0
1,"С вами снова я, Денис.",0
2,Рад вас приветствовать на лекции.,0
3,Сегодня мы будем говорить о массивах.,0
4,На прошлой лекции мы познакомились с языком пр...,0
...,...,...
492,Первый аспект применения языка – это именовани...,1
493,"Следующий аспект, где английский может существ...",1
494,Третий аспект применения английского – это пои...,1
495,"Ну и четвертый аспект применения английского, ...",1


In [22]:
df = shuffle(df)
df

,text,target
444,"На этом, друзья, я с вами прощаюсь.",0
232,"Итак, сохраним изменения и запустим еще раз.",0
191,Однако при переносе блок-схемы на конкретный я...,0
340,"Соответственно, мы не знаем, на сколько итерац...",0
298,Это while.,0
...,...,...
48,Для этого выведем каждый его элемент на экран ...,0
260,"То есть wait1, wait2 и так далее.",0
312,"Давайте посмотрим, как работает цикл в приведе...",0
207,Перейдем в каталог с заданием 3 и создадим там...,0


In [37]:
train_df, test_df = model_selection.train_test_split(df, test_size=0.2, random_state=None, shuffle=True, stratify=df['target'])
train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

In [38]:
train_text = train_df['text'].astype('str')
train_labels = train_df['target']
test_text = test_df['text'].astype('str')
test_labels = test_df['target']

In [39]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [40]:
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased-sentence', num_labels=2, ignore_mismatched_sizes=True).to("cuda")
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
seq_len_train = [len(str(i).split()) for i in train_df['text']]
seq_len_test = [len(str(i).split()) for i in test_df['text']]
max_seq_len = max(max(seq_len_test), max(seq_len_train))
max_seq_len

80

In [42]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.values,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)

In [43]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train, train_labels)
test_dataset = Data(tokens_test, test_labels)

In [44]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'F1': f1}

In [45]:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 50, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=21)

In [46]:
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = train_dataset,
                  compute_metrics = compute_metrics)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [47]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.370800,0.300249,0.000000
2,0.293000,0.198069,0.260870
3,0.202500,0.103102,0.769231
4,0.095500,0.027925,0.974359
5,0.042200,0.017921,0.974359
6,0.024200,0.008333,0.987342
7,0.007200,0.001852,1.000000
8,0.001400,0.000882,1.000000
9,0.001000,0.003708,0.987654
10,0.000900,0.000563,1.000000


TrainOutput(global_step=2500, training_loss=0.020943645702116192, metrics={'train_runtime': 661.7676, 'train_samples_per_second': 29.995, 'train_steps_per_second': 3.778, 'total_flos': 816055382640000.0, 'train_loss': 0.020943645702116192, 'epoch': 50.0})

In [48]:
model_path = "fine-tune-bert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('fine-tune-bert/tokenizer_config.json',
 'fine-tune-bert/special_tokens_map.json',
 'fine-tune-bert/vocab.txt',
 'fine-tune-bert/added_tokens.json')

In [49]:
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

In [50]:
print(classification_report(test_labels, pred))
print(f1_score(test_labels, pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96        90
           1       0.75      0.30      0.43        10

    accuracy                           0.92       100
   macro avg       0.84      0.64      0.69       100
weighted avg       0.91      0.92      0.90       100

0.4285714285714285


In [55]:
model_path = "fine-tune-bert"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)
max_seq_len = 80

In [58]:
texts = 'Питон - это язык програмирования.'

In [67]:
texts = 'Сегодня на лекции мы поговорим про програмирование'

In [75]:
def classify(text):
    '0 - не термин'
    '1 - термин'
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    return int(logits.argmax())

In [70]:
with open('2.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [76]:
sentences = split_into_sentences(text)

for sentence in sentences:
    class_ = classify(sentence)
    if class_:
        print(sentence)
        print()

Под рекурсией в широком смысле мы понимаем описание объекта или процесса, который находится в этом же самом объекте или же процессе.

Описание объекта, например, одна из матрешек, который находится в этом же объекте, в некоторой более крупной матрешке.

Третий пример похож на первый.

Первая ветка, вторая и третья в самом низу.

И эта область памяти называется стэк.

Дерево это один из способов хранения и упорядочивания данных, наряду с массивом.

Второй пример – это алгоритм быстрой сортировки, также его называют Quick Sort.

Рекурсия здесь является одним из основных компонентов.

В этой формуле также используется функция факториал.

Второй компонент – это условия остановки рекурсии.

Для него восклицательный знак является несколько другой операцией.

Факториал имеет более простое решение с применением цикла.

Напомню, for мы используем, когда количество итераций у нас заранее известно.

Кроме того, итеративное решение не опасно переполнением стека, которое может возникать при рекурси